In [1]:
# https://github.com/jukyellow/algoTrade/blob/master/ch06/6.1%20ETFs%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EC%A3%BC%EA%B0%80%20%EB%B0%A9%ED%96%A5%20%EC%98%88%EC%B8%A1.ipynb

In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn import svm

import seaborn as sns; sns.set()

In [3]:
pd.set_option('display.max_columns', 30)
print(os.getcwd())

/content


In [4]:
SPY = pd.read_csv('./SPY.csv',parse_dates=['Dates'], index_col = 'Dates')
GLD = pd.read_csv('./GLD.csv',parse_dates=['Dates'], index_col = 'Dates')
FXY = pd.read_csv('./FXY.csv',parse_dates=['Dates'], index_col = 'Dates')
T10Y2Y = pd.read_csv('./T10Y2Y.csv',parse_dates=['Dates'], index_col = 'Dates')
TED = pd.read_csv('./TED.csv',parse_dates=['Dates'], index_col = 'Dates')
USO = pd.read_csv('./USO.csv',parse_dates=['Dates'], index_col = 'Dates')
UUP = pd.read_csv('./UUP.csv',parse_dates=['Dates'], index_col = 'Dates')
VIX = pd.read_csv('./VIX.csv',parse_dates=['Dates'], index_col = 'Dates')
VWO = pd.read_csv('./VWO.csv',parse_dates=['Dates'], index_col = 'Dates')

In [5]:
t1 = SPY.merge(GLD, left_index=True,
              right_index=True, how = 'outer',
              suffixes=('_SPY', '_GLD'))
list_df = ['FXY', 'T10Y2Y', 'TED', 'USO', 'UUP', 'VIX', 'VWO']
for x in list_df:
    t1 = t1.merge(globals()[x].add_suffix('_'+str(x)),
                 left_index=True, right_index=True,
                 how='outer')

In [6]:
print(len(t1))
df = t1.dropna()
print(len(df))
print(df.tail())

5309
2771
            CLOSE_SPY    OPEN     HIGH    LOW       VOLUME  CLOSE_GLD  \
Dates                                                                   
2018-12-20     247.17  249.86  251.620  245.0  252053406.0     119.24   
2018-12-21     240.70  246.74  249.710  240.0  255345594.0     118.72   
2018-12-24     234.34  239.04  240.836  234.0  147311594.0     120.02   
2018-12-27     248.07  242.57  248.290  239.0  186267297.0     120.57   
2018-12-28     247.75  249.58  251.400  246.0  153100188.0     121.06   

            CLOSE_FXY  CLOSE_T10Y2Y  CLOSE_TED  CLOSE_USO  CLOSE_UUP  \
Dates                                                                  
2018-12-20      85.87        1.7807       0.48       9.72      25.77   
2018-12-21      85.87        1.7651       0.48       9.57      25.94   
2018-12-24      86.55        1.7505       0.40       9.29      25.55   
2018-12-27      86.00        1.7581       0.44       9.62      25.57   
2018-12-28      86.63        1.7320       0.45

In [7]:
export_csv = df.to_csv (r'./ETFs_main.csv', index = True, header=True)
df = pd.read_csv('./ETFs_main.csv')
df.head()

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.20,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.07,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.42,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.79,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.95,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960


In [8]:
#기술적 지표 만들기
def moving_average(df, n):
    MA = pd.Series(df['CLOSE_SPY'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
    df = df.join(MA)
    return df

def volume_moving_average(df, n):
    VMA = pd.Series(df['VOLUME'].rolling(n, min_periods=n).mean(), name='VMA_' + str(n))
    df = df.join(VMA)
    return df

def relative_strength_index(df, n):
    """Calculate Relative Strength Index(RSI) for given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    i = 0
    UpI = [0]
    DoI = [0]    
    while i + 1 <= df.index[-1]:
        UpMove = df.loc[i + 1, 'HIGH'] - df.loc[i, 'HIGH']
        DoMove = df.loc[i, 'LOW'] - df.loc[i + 1, 'LOW']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
    RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
    df = df.join(RSI)
    return df

In [9]:
# 이동평균, 거래량 이동평균, 시장 강도 지수(RSI)
df = moving_average(df, 45) #60일평균->실제 영업일 45일
df = volume_moving_average(df, 45)
df = relative_strength_index(df, 14) #21 or 14일
df.tail()

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
2766,2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.18,269.767778,1.240592e+08,0.172439
2767,2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.87,269.018889,1.274610e+08,0.142578
2768,2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.32,267.995333,1.281067e+08,0.115003
2769,2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.90,267.275778,1.297876e+08,0.307099
2770,2018-12-28,247.75,249.58,251.400,246.0,153100188.0,121.06,86.63,1.7320,0.45,9.53,25.50,28.34,38.17,266.639111,1.301996e+08,0.372653


In [10]:
df = df.set_index('Dates')
df = df.dropna() #결측치 제거
len(df)

2727

In [11]:
df['target'] = df['CLOSE_SPY'].pct_change() #일별 수익률 계산하기위해 변화율 계산
print(df['target'].describe())
df.tail()

count    2726.000000
mean        0.000270
std         0.013026
min        -0.098448
25%        -0.004320
50%         0.000545
75%         0.005791
max         0.128249
Name: target, dtype: float64


,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14,target
Dates,,,,,,,,,,,,,,,,,
2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.18,269.767778,1.240592e+08,0.172439,-0.016278
2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.87,269.018889,1.274610e+08,0.142578,-0.026176
2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.32,267.995333,1.281067e+08,0.115003,-0.026423
2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.90,267.275778,1.297876e+08,0.307099,0.058590
2018-12-28,247.75,249.58,251.400,246.0,153100188.0,121.06,86.63,1.7320,0.45,9.53,25.50,28.34,38.17,266.639111,1.301996e+08,0.372653,-0.001290


In [12]:
# 1번 Case 수수료 0으로 했을 때
# df['target'] = np.where(df['target'] > 0.0005 , 1, -1)
df['target'] = np.where(df['target'] > 0 , 1, -1) # 0보다 크면 1, 작으면 -1
df['target'].value_counts()

 1    1471
-1    1256
Name: target, dtype: int64

In [13]:
df['target'] = df['target'].shift(-1) # 당일 데이터를 활용해 다음날 예측, 다음날 트렌드를 한행 앞으로 당김
df = df.dropna()
len(df)

2726

In [14]:
df['target'] = df['target'].astype(np.int64) # 정수처리
df.tail()

,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14,target
Dates,,,,,,,,,,,,,,,,,
2018-12-19,251.26,255.17,259.400,249.0,214992797.0,117.43,84.83,1.7824,0.44,10.02,25.97,25.58,37.89,270.407333,1.225288e+08,0.201733,-1
2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.18,269.767778,1.240592e+08,0.172439,-1
2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.87,269.018889,1.274610e+08,0.142578,-1
2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.32,267.995333,1.281067e+08,0.115003,1
2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.90,267.275778,1.297876e+08,0.307099,-1


In [15]:
y_var = df['target']
# 예측에 사용하지 않는 변수 삭제
x_var = df.drop(['target', 'OPEN','HIGH','LOW', 'VOLUME','CLOSE_SPY'], axis=1)
x_var.head()

,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO,MA_45,VMA_45,RSI_14
Dates,,,,,,,,,,,
2007-04-27,67.56,83.7300,2.4474,0.55,51.84,24.54,12.45,41.750,143.551556,1.106696e+08,0.670018
2007-04-30,67.09,83.7166,2.4361,0.57,51.24,24.49,14.22,40.935,143.601556,1.116466e+08,0.531751
2007-05-02,66.66,83.3800,2.4366,0.59,49.59,24.66,13.08,42.020,143.680667,1.121613e+08,0.554050
2007-05-03,67.49,83.1100,2.4346,0.60,49.28,24.69,13.09,42.435,143.780222,1.123421e+08,0.601028
2007-05-04,68.19,83.2300,2.4006,0.60,48.30,24.60,12.91,42.595,143.905111,1.128853e+08,0.665987


In [16]:
up=df[df['target']==1].target.count()
total=df.target.count()
print('up/down ratio: {0:.2f}'.format((up/total)))

up/down ratio: 0.54


In [17]:
#훈련 세트와 테스트 세트를 나눈다.
#shuffle=False을 설정해 기간이 섞이지 않도록 만든다.
X_train, X_test, y_train, y_test = train_test_split(x_var, 
                                                    y_var, 
                                                    test_size=0.3, 
                                                    shuffle=False, 
                                                    random_state=3)

print('X_train:',X_train.shape, ' ,y_train:', y_train.shape)
print('X_test:',X_test.shape, ' ,y_test:', y_test.shape)


#훈련 세트와 테스트 세트에 포함된 양성 샘플(up) 비율을 확인한다.
train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts()/train_count)
print('test set label ratio')
print(y_test.value_counts()/test_count)

X_train: (1908, 11)  ,y_train: (1908,)
X_test: (818, 11)  ,y_test: (818,)
train set label ratio
 1    0.543501
-1    0.456499
Name: target, dtype: float64
test set label ratio
 1    0.530562
-1    0.469438
Name: target, dtype: float64


In [18]:
def get_confusion_matrix(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('confusion matrix')
    print('accuracy:{0:.4f},precision:{1:.4f},recall:{2:.4f},F1:{3:.4f},ROC AUC score:{4:.4f}'.format(accuracy, precision, recall, f1,roc_score))

In [19]:
# 수익률 0이였을 때
# 1-1번째 모델링(XGBoost Classifier)

# XGBClassifier 참고
# https://joyfuls.tistory.com/63
# https://statkclee.github.io/model/model-python-xgboost-hyper.html
xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3) # 의사결정트리400개
xgb_dis.fit(X_train, y_train)
xgb_pred = xgb_dis.predict(X_test)

print(xgb_dis)
print(xgb_dis.predict_proba(X_test))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
[[0.7526408  0.24735922]
 [0.9640353  0.03596473]
 [0.6700425  0.32995752]
 ...
 [0.08560187 0.91439813]
 [0.07786518 0.9221348 ]
 [0.10759658 0.8924034 ]]


In [20]:
# 훈련 data accurary
print(xgb_dis.score(X_train, y_train)) 
print()
# 테스트 data 결과
get_confusion_matrix(y_test, xgb_pred)

0.8763102725366876

confusion matrix
accuracy:0.4914,precision:0.5242,recall:0.4493,F1:0.4839,ROC AUC score:0.4942


In [21]:
# 1-2번째 모델링(Random Forest Classifier)

n_estimators = range(10,200,10)
params = {
    'bootstrap': [True],
    'n_estimators':n_estimators,
    'max_depth':[4,6,8,10,12],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_features':[4]
}

In [22]:
my_cv = TimeSeriesSplit(n_splits=5).split(X_train)
my_cv

<generator object TimeSeriesSplit.split at 0x7f7690021e08>

In [23]:
# 여러가지 값을 돌아가면서 설정된 페라미터로 테스트
clf = GridSearchCV(RandomForestClassifier(), params, cv=my_cv, n_jobs=-1)
clf 

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x7f7690021e08>,
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_w...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
  

In [24]:
import time
start_time = time.time()
clf.fit(X_train, y_train)
print(time.time() - start_time," seconds consumed")


1973.473917722702  seconds consumed


In [25]:

## Random Forest에서 best param
print('best parameter:\n', clf.best_params_)
print('best prediction:{0:.4f}'.format(clf.best_score_))

# 테스트 data 결과
pred_con = clf.predict(X_test)
accuracy_con = accuracy_score(y_test, pred_con)
print('accuracy:{0:.4f}'.format(accuracy_con))
get_confusion_matrix(y_test, pred_con)

best parameter:
 {'bootstrap': True, 'max_depth': 4, 'max_features': 4, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 150}
best prediction:0.5566
accuracy:0.4976
confusion matrix
accuracy:0.4976,precision:0.5370,recall:0.3848,F1:0.4483,ROC AUC score:0.5049


In [26]:
# 2번 0.05% 수익률 적용시.
df = pd.read_csv('./ETFs_main.csv')
df = moving_average(df, 45)
df = volume_moving_average(df, 45)
df = relative_strength_index(df, 14)
df = df.set_index('Dates')
df = df.dropna()
df['target'] = df['CLOSE_SPY'].pct_change()

In [27]:
df['target'] = np.where(df['target'] > 0.0005 , 1, -1) # 수익률 0.05% 이상시 오른것으로 간주!
# df['target'] = np.where(df['target'] > 0 , 1, -1)
print(df['target'].value_counts()) # 상승/하락 확률이 거의 동일해짐

df['target'] = df['target'].shift(-1)
df = df.dropna()
len(df)

df['target'] = df['target'].astype(np.int64)
y_var = df['target']
# 예측에 사용하지 않는 변수 삭제
x_var = df.drop(['target', 'OPEN','HIGH','LOW', 'VOLUME','CLOSE_SPY'], axis=1) 
up=df[df['target']==1].target.count()
total=df.target.count()
print('up/down ratio: {0:.2f}'.format((up/total)))

 1    1375
-1    1352
Name: target, dtype: int64
up/down ratio: 0.50


In [28]:
#훈련 세트와 테스트 세트를 나눈다.
#shuffle=False을 설정해 기간이 섞이지 않도록 만든다.
X_train, X_test, y_train, y_test = train_test_split(x_var, 
                                                    y_var, 
                                                    test_size=0.3, 
                                                    shuffle=False, 
                                                    random_state=3)

#훈련 세트와 테스트 세트에 포함된 양성 샘플(up) 비율을 확인한다.
train_count = y_train.count()
test_count = y_test.count()

print('train set label ratio')
print(y_train.value_counts()/train_count)
print('test set label ratio')
print(y_test.value_counts()/test_count)

train set label ratio
 1    0.509958
-1    0.490042
Name: target, dtype: float64
test set label ratio
-1    0.508557
 1    0.491443
Name: target, dtype: float64


In [29]:
xgb_dis = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb_dis.fit(X_train, y_train)
xgb_pred = xgb_dis.predict(X_test)
print(xgb_dis.score(X_train, y_train))
get_confusion_matrix(y_test, xgb_pred)

0.8726415094339622
confusion matrix
accuracy:0.5012,precision:0.4920,recall:0.4602,F1:0.4756,ROC AUC score:0.5005


In [30]:
n_estimators = range(10,200,10)

params = {
    'bootstrap': [True],
    'n_estimators':n_estimators,
    'max_depth':[4,6,8,10,12],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_features':[4]
}

my_cv = TimeSeriesSplit(n_splits=5).split(X_train)
clf = GridSearchCV(RandomForestClassifier(), params, cv=my_cv, n_jobs=-1)

In [ ]:
start_time = time.time()
clf.fit(X_train, y_train)
print(time.time() - start_time," seconds consumed")

In [ ]:
## Random Forest에서 best param
print('best parameter:\n', clf.best_params_)
print('best prediction:{0:.4f}'.format(clf.best_score_))

pred_con = clf.predict(X_test)
accuracy_con = accuracy_score(y_test, pred_con)
print('accuracy:{0:.4f}'.format(accuracy_con))
get_confusion_matrix(y_test, pred_con)